# 워드엠베딩_워드투벡터
[워드엠베딩](https://en.wikipedia.org/wiki/Word_embedding)은 단어를 벡터로 변환해 표현하는 것을 의미합니다.  
서로 가까운 의미의 단어는 벡터 공간 상에서 서로 가깝게, 먼 단어는 멀게 표현합니다.  
<br>
이렇게 표현하는 경우 어떤 의미가 있는지에 대한 사례 [기사](http://www.hani.co.kr/arti/science/technology/900874.html)<br>
가장 반가운 특징은 사람처럼 의미를 이해하는 듯이 작동한다는 점일 것입니다.<br>
<br>
워드투벡(Word2vec)는 워드엠베딩의 한 방법입니다.  
구글에서 일하던 토머스 미콜로프(Thomas Mikolov)가 2013년 발표하였습니다.  
우선 워드투벡터에 대한 이론은 [딥 러닝을 이용한 자연어처리 입문](https://wikidocs.net/22644)을 참조합니다.<br>
  
실습 역시 위 교재의 예를 활용합니다.  
라이브러리는 토픽모델링과 마찬가지로 Gensim을 활용합니다.([documentation](https://radimrehurek.com/gensim/models/word2vec.html))

In [ ]:
import urllib.request
from konlpy.tag import Okt
from gensim.models.word2vec import Word2Vec
import pandas as pd
import matplotlib.pyplot as plt

워드투벡 모델이 학습할 훈련 데이터는 웹에서 가져옵니다.  
앞서 실습에서 활용한 **빅카인즈** 데이터 경우 문서의 숫자가 필요 학습량에 비해 좀 적습니다.

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")

In [ ]:
train_data = pd.read_table('ratings.txt') 

In [ ]:
train_data[:5] # 상위 5개 출력

In [ ]:
print(len(train_data)) # 리뷰 개수 출력

## 전처리

In [ ]:
# NULL 값 존재 유무
print(train_data.isnull().values.any())

In [ ]:
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인

In [ ]:
print(len(train_data)) # 리뷰 개수 출력

In [ ]:
# 정규 표현식을 통한 한글 외 문자 제거
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

In [ ]:
# 불용어 정의
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [ ]:
# 형태소 분석기 OKT를 사용한 토큰화 작업 (긴 시간 소요)
okt = Okt()
tokenized_data = []
for sentence in train_data['document']:
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    tokenized_data.append(temp_X)

In [ ]:
# 리뷰 길이 분포 확인
print('리뷰의 최대 길이 :',max(len(l) for l in tokenized_data))
print('리뷰의 평균 길이 :',sum(map(len, tokenized_data))/len(tokenized_data))
plt.hist([len(s) for s in tokenized_data], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

## 모델링

In [ ]:
from gensim.models import Word2Vec
model = Word2Vec(sentences = tokenized_data, vector_size = 100, window = 5, min_count = 5, workers = 4, sg = 0) #100차원

In [ ]:
# 완성된 임베딩 매트릭스의 크기 확인
model.wv.vectors.shape

In [ ]:
print(model.wv.most_similar("김혜수"))

In [ ]:
print(model.wv.most_similar("히어로"))

### 모델 저장

In [ ]:
model.save("naver_mv.model")

## 미리 학습된 모델 사용하기
대량의 텍스트 데이터로 미리 학습된 워드투벡 모델을 로딩하면 학습 과정 없이 활용할 수 있습니다.  
단어 의미를 이해하는 듯한 모델은 여러가지 용도로 쓰일 수 있을 것입니다.  
박규병님이 [공개](https://github.com/Kyubyong/wordvectors)한 [모델](https://drive.google.com/file/d/0B0ZXk88koS2KbDhXdWg1Q2RydlU/view)은 위키피디아의 30개 언어 이상의 데이터를 학습하였습니다.<br>
<br>
ko.zip 다운로드 뒤 압축을 실습 폴더에 풀고 ko.bin 파일을 로드

In [ ]:
import gensim
model = gensim.models.Word2Vec.load('ko.bin') # gensim 최신 버전 v4.0에서 에러 발생
#model= gensim.models.KeyedVectors.load("ko.bin", mmap='r') 역시 에러 발생

In [ ]:
result = model.wv.most_similar("강아지")
print(result)